In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from implicit.evaluation import mean_average_precision_at_k
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from lightfm.cross_validation import random_train_test_split

import warnings
warnings.filterwarnings('ignore')
from lightfm import LightFM

C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


import error: No module named 'implicit.gpu._cuda'


c:\users\iltya\appdata\local\programs\python\python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
X = pd.read_csv('train_df.tsv.gz', sep = '\t', index_col=0).drop_duplicates()
X.head(5)

,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date
0,00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...,A,NaN
1,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,b935c3390d82df612de19591d2dd16e1eff8e660746bb3...,A,NaN
2,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...,A,NaN
3,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...,A,NaN
4,097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...,Читать внимательно!!! Причины попадания в чёрн...,1035,1,7,1.041779e+10,NaN,FAN_CLUB,BLOG,780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...,A,NaN


In [3]:
# кодируем id-шники
from sklearn.preprocessing import LabelEncoder

encoder_com_id = LabelEncoder()
encoder_com_id.fit(X.community_id)
X["community_id"] = encoder_com_id.transform(X["community_id"])
encoder_usr_id = LabelEncoder()
encoder_usr_id.fit(X.customer_id)
X["customer_id"] = encoder_usr_id.transform(X["customer_id"])

In [4]:
community_features = X.groupby("community_id").head(1)

In [5]:
community_features

,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date
0,486,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,908981,A,NaN
1,3578,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,1134537,A,NaN
4,5627,Читать внимательно!!! Причины попадания в чёрн...,1035,1,7,1.041779e+10,NaN,FAN_CLUB,BLOG,734661,A,NaN
5,7358,Здесь вы сможете увидеть все спортивные новост...,6573,2,7,4.275314e+10,209.0,NaN,NaN,118010,A,NaN
6,12246,"Все про охоту,природу и рыбалку в Сибири и за...",3656,1,7,1.040759e+10,320.0,NaN,NaN,1405863,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15585934,42613,"🌺""Одна из самых больших удач в жизни человека ...",114,2,7,NaN,NaN,GOS_KINDERGARTEN,GOS_ORGANIZATIONS,533361,I,NaN
15587787,63961,1 сентября исполняется 35 лет Александровскому...,398,2,7,1.040711e+10,NaN,NaN,NaN,1500072,P,NaN
15587902,76648,NaN,150,2,7,NaN,NaN,GOS_KINDERGARTEN,GOS_ORGANIZATIONS,292591,P,NaN
15590003,81188,NaN,131,1,7,NaN,NaN,NaN,NaN,215592,P,NaN


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [7]:
vec = TfidfVectorizer(max_features=300)
tfidf_res = vec.fit_transform(community_features.description.fillna(""))

In [8]:
columns_to_encode = ["business_category", "business_parent", "themeid", "region_id", "type", "status"]
results_encoding = [tfidf_res]

for column_for_encoding in columns_to_encode:
    encoder = OneHotEncoder(min_frequency=50)
    nan_value_replacer = "unkonown" if community_features[column_for_encoding].dtype == np.dtype('O') else -1

    result_encoding = encoder.fit_transform(community_features[column_for_encoding].fillna(nan_value_replacer).values.reshape(-1, 1))
    print(f"ersult encoding shape: {result_encoding.shape}")
    results_encoding.append(result_encoding)

ersult encoding shape: (149114, 122)
ersult encoding shape: (149114, 21)
ersult encoding shape: (149114, 156)
ersult encoding shape: (149114, 159)
ersult encoding shape: (149114, 2)
ersult encoding shape: (149114, 5)


In [9]:
results_encoding.append(community_features[["customers_count", "messages_count"]].values)

In [10]:
import scipy
all_features = scipy.sparse.hstack(results_encoding)
all_features.shape

(149114, 767)

In [11]:
df_reduced = X[['community_id', 'customer_id']].drop_duplicates()

In [12]:
data_full = csr_matrix((np.ones((df_reduced.shape[0],)).astype(float),
                        (df_reduced['customer_id'].values,
                         df_reduced['community_id'].values)))

In [13]:
from lightfm import  LightFM
model = LightFM(loss='warp', random_state=42)

In [ ]:
model.fit(data_full, epochs=3, verbose=True, num_threads=1)

In [2]:
data_full

NameError: name 'data_full' is not defined